In [3]:
import pandas as pd

def convert_allocations():
    input_file = 'allocations.csv'
    
    try:
        # Read the CSV file
        df = pd.read_csv(input_file)
        print(f"Read {len(df)} rows from {input_file}")

        # Filter out rows where LOC_CODE is 'UNFIT'
        # (This removes the error row shown in your example)
        df = df[df['LOC_CODE'] != 'UNFIT'].copy()

        # --- Perform Calculations ---

        # 1. Derive LOCATION_TYPE from LOC_CODE (e.g., 'A1' from 'A1-00001')
        df['LOCATION_TYPE'] = df['LOC_CODE'].astype(str).str.split('-').str[0]

        # 2. Set MAX_UNITS (Assuming equal to allocated qty based on provided example)
        df['MAX_UNITS'] = df['QTY_ALLOCATED']

        # 3. Calculate Stored Volume in Cubic Meters
        # Formula: (L * W * H * Qty) / 1,000,000,000 (conversion from mm3 to m3)
        stored_vol_mm3 = (
            df['ORIENT_X_MM'] * 
            df['ORIENT_Y_MM'] * 
            df['ORIENT_Z_MM'] * 
            df['QTY_ALLOCATED']
        )
        df['STORED_VOL_M3'] = stored_vol_mm3 / 1_000_000_000

        # 4. Back-calculate Total Location Volume based on Utilization %
        # Formula: (Stored Vol / Utilization) * 100
        # We use a lambda to avoid DivisionByZero errors if utilization is 0
        df['LOCATION_VOL_MM3'] = df.apply(
            lambda x: (x['STORED_VOL_M3'] * 1_000_000_000 / x['UTILIZATION_PCT'] * 100) 
            if x['UTILIZATION_PCT'] > 0 else 0, axis=1
        )

        # 5. Calculate Location Volume in Cubic Meters
        df['LOCATION_VOL_M3'] = df['LOCATION_VOL_MM3'] / 1_000_000_000

        # --- Formatting ---

        # Rename LOC_CODE to loc_inst_code
        df.rename(columns={'LOC_CODE': 'loc_inst_code'}, inplace=True)

        # Round values for cleaner output
        df['LOCATION_VOL_MM3'] = df['LOCATION_VOL_MM3'].round(1)
        df['LOCATION_VOL_M3'] = df['LOCATION_VOL_M3'].round(4)
        df['STORED_VOL_M3'] = df['STORED_VOL_M3'].round(4)

        # Define the exact column order required
        target_columns = [
            'loc_inst_code', 'LOCATION_TYPE', 'ITEM_ID', 'QTY_ALLOCATED', 
            'MAX_UNITS', 'GRID_X', 'GRID_Y', 'GRID_Z', 
            'FULL_LAYERS', 'PARTIAL_UNITS', 
            'ORIENT_X_MM', 'ORIENT_Y_MM', 'ORIENT_Z_MM', 
            'LOCATION_VOL_MM3', 'LOCATION_VOL_M3', 'STORED_VOL_M3', 
            'UTILIZATION_PCT'
        ]

        # Select only these columns
        df_final = df[target_columns]

        # Overwrite the original file
        df_final.to_csv(input_file, index=False)
        
        print(f"Conversion successful. {len(df_final)} rows written to {input_file}")

    except FileNotFoundError:
        print(f"Error: {input_file} not found.")
    except Exception as e:
        print(f"An error occurred: {e}")

if __name__ == "__main__":
    convert_allocations()

Read 11 rows from allocations.csv
Conversion successful. 10 rows written to allocations.csv
